> The most common usage of `hover` is through built-in `recipe`s like in the quickstart.
>
> :ferris_wheel: Let's explore another `recipe` -- an active learning example.

-   <details open><summary>Dependencies for {== local environments ==}</summary>
    When you run the code locally, you may need to install additional packages.

    To run the text embedding code on this page, you need:
```shell
    pip install spacy
    python -m spacy download en_core_web_md
```

    To render `bokeh` plots in Jupyter, you need:
```shell
    pip install jupyter_bokeh
```

    If you are using JupyterLab older than 3.0, use this instead ([reference](https://pypi.org/project/jupyter-bokeh/)):
```shell
    jupyter labextension install @jupyter-widgets/jupyterlab-manager
    jupyter labextension install @bokeh/jupyter_bokeh
```

</details>

## **Fundamentals**

Hover `recipe`s are functions that take a `SupervisableDataset` and return an annotation interface.

The `SupervisableDataset` is assumed to have some data and embeddings.

## **Recap: Data & Embeddings**

Let's preprare a dataset with embeddings. This is almost the same as in the [quickstart](../t0-quickstart/):

In [1]:
from hover.core.dataset import SupervisableTextDataset
import pandas as pd

raw_csv_path = "https://raw.githubusercontent.com/phurwicz/hover-gallery/main/0.5.0/20_newsgroups_raw.csv"
train_csv_path = "https://raw.githubusercontent.com/phurwicz/hover-gallery/main/0.5.0/20_newsgroups_train.csv"

# for fast, low-memory demonstration purpose, sample the data
df_raw = pd.read_csv(raw_csv_path).sample(400)
df_raw["SUBSET"] = "raw"
df_train = pd.read_csv(train_csv_path).sample(400)
df_train["SUBSET"] = "train"
df_dev = pd.read_csv(train_csv_path).sample(100)
df_dev["SUBSET"] = "dev"
df_test = pd.read_csv(train_csv_path).sample(100)
df_test["SUBSET"] = "test"

# build overall dataframe and ensure feature type
df = pd.concat([df_raw, df_train, df_dev, df_test])
df["text"] = df["text"].astype(str)

# this class stores the dataset throught the labeling process
dataset = SupervisableTextDataset.from_pandas(df, feature_key="text", label_key="label")

🔵 SupervisableTextDataset: Initializing...

🔵 SupervisableTextDataset: Deduplicating...

🔵 SupervisableTextDataset: --subset raw rows: 400 -> 372.

🔵 SupervisableTextDataset: --subset train rows: 400 -> 384.

🔵 SupervisableTextDataset: --subset dev rows: 100 -> 97.

🔵 SupervisableTextDataset: --subset test rows: 100 -> 99.

🟢 SupervisableTextDataset: Set up label encoder/decoder with 20 classes.

🟢 SupervisableTextDataset: Population updater: latest population with 20 classes.

🟢 SupervisableTextDataset: SupervisableTextDataset: finished initialization.

<br>

In [2]:
import spacy
import re
from functools import lru_cache

# use your preferred embedding for the task
nlp = spacy.load("en_core_web_md")

# raw data (str in this case) -> np.array
@lru_cache(maxsize=int(1e+4))
def vectorizer(text):
    clean_text = re.sub(r"[\s]+", r" ", str(text))
    return nlp(clean_text, disable=nlp.pipe_names).vector

text = dataset.dfs["raw"].loc[0, "text"]
vec = vectorizer(text)
print(f"Text: {text}")
print(f"Vector shape: {vec.shape}")

Text: I have a chance to buy a used PS/2 Appletalk card to create a network with my home machines.  However, the guy who has the card tells me there's a DB-9 or DB-15 (can't remember now) on the back of the card, rather than the 8-pin (or 4-pin) mini-din that I expect.  This sounds more like a Thicknet ethernet card.  Should there be a transciever on it, like on the Quadras?  What would be a reasonable price to expect to pay for one of these cards, keeping in mind that it's Micro-Channel Architecture, which means take your best guess and double it. Email replies would be appreciated, to here or to rrr@ideas.com Thanks. [RICHR]
Vector shape: (300,)


<br>

In [3]:
# any kwargs will be passed onto the corresponding reduction
# for umap: https://umap-learn.readthedocs.io/en/latest/parameters.html
# for ivis: https://bering-ivis.readthedocs.io/en/latest/api.html
reducer = dataset.compute_nd_embedding(vectorizer, "umap", dimension=2)

Vectorizing: 100%|██████████| 952/952 [00:02<00:00, 351.18it/s]


🔵 SupervisableTextDataset: Fit-transforming UMAP on 853 samples...

🔵 SupervisableTextDataset: Transforming UMAP on 99 samples...

🟢 SupervisableTextDataset: Computed 2-d embedding in columns ['embed_2d_0', 'embed_2d_1']

<br>

## **Recipe-Specific Ingredient**

Each recipe has different functionalities and potentially different signature.

To utilize active learning, we need to specify how to get a model in the loop.

`hover` considers the `vectorizer` as a "frozen" embedding and follows up with a neural network, which infers its own dimensionality from the vectorizer and the output classes.

-   This architecture named [`VectorNet`](../../reference/core-neural/#hover.core.neural.VectorNet) is the (default) basis of active learning in `hover`.

-   <details open><summary>Custom models</summary>
    It is possible to use a model other than `VectorNet` or its subclass.

    You will need to implement the following methods with the same signatures as `VectorNet`:

    -   [`train`](../../reference/core-neural/#hover.core.neural.VectorNet.train)
    -   [`save`](../../reference/core-neural/#hover.core.neural.VectorNet.save)
    -   [`predict_proba`](../../reference/core-neural/#hover.core.neural.VectorNet.predict_proba)
    -   [`prepare_loader`](../../reference/core-neural/#hover.core.neural.VectorNet.prepare_loader)
    -   [`manifold_trajectory`](../../reference/core-neural/#hover.core.neural.VectorNet.manifold_trajectory)
</details>

In [4]:
from hover.core.neural import VectorNet
from hover.utils.common_nn import LogisticRegression

# Create a model with vectorizer-NN architecture.
# model.pt will point to a PyTorch state dict (to be created)
# the label classes in the dataset can change, and vecnet can adjust to that
vecnet = VectorNet(vectorizer, LogisticRegression, "model.pt", dataset.classes)

# predict_proba accepts individual strings or list
# text -> vector -> class probabilities
# if no classes right now, will see an empty list
print(vecnet.predict_proba(text))
print(vecnet.predict_proba([text]))

🟢 VectorNet: reset neural net: in 300 out 20.

[4.5319758e-02 3.7731754e-04 7.1024925e-01 2.3023796e-04 3.6372606e-02
 2.8025556e-02 2.2987858e-03 8.1140507e-04 1.1964727e-02 4.7368396e-04
 5.6628410e-02 4.1806232e-03 2.2704415e-02 8.2551029e-05 9.1974456e-03
 9.2212635e-04 6.6293120e-02 3.3224770e-03 4.8326887e-04 6.2286876e-05]
[[4.5319758e-02 3.7731754e-04 7.1024925e-01 2.3023796e-04 3.6372606e-02
  2.8025556e-02 2.2987858e-03 8.1140507e-04 1.1964727e-02 4.7368396e-04
  5.6628410e-02 4.1806232e-03 2.2704415e-02 8.2551029e-05 9.1974456e-03
  9.2212635e-04 6.6293120e-02 3.3224770e-03 4.8326887e-04 6.2286876e-05]]


Note how the callback dynamically takes `dataset.classes`, which means the model architecture will adapt when we add classes during annotation.

## :sparkles: **Apply Labels**

Now we invoke the `active_learning` recipe.

-   <details open><summary>Tips: how recipes work programmatically</summary>
    In general, a `recipe` is a function taking a `SupervisableDataset` and other arguments based on its functionality.

    Here are a few common recipes:

    === "active_learning"

        ::: hover.recipes.experimental.active_learning
            rendering:
              show_root_heading: false
              show_root_toc_entry: false

    === "simple_annotator"

        ::: hover.recipes.stable.simple_annotator
            rendering:
              show_root_heading: false
              show_root_toc_entry: false

    === "linked_annotator"

        ::: hover.recipes.stable.linked_annotator
            rendering:
              show_root_heading: false
              show_root_toc_entry: false

    The recipe returns a `handle` function which `bokeh` can use to visualize an annotation interface in multiple settings.
</details>

In [5]:
from hover.recipes.experimental import active_learning

interactive_plot = active_learning(dataset, vecnet)

# ---------- NOTEBOOK MODE: for your actual Jupyter environment ---------
# this code will render the entire plot in Jupyter
# from bokeh.io import show, output_notebook
# output_notebook()
# show(interactive_plot, notebook_url='https://localhost:8888')

-   <details open><summary>Tips: annotation interface with multiple plots</summary>
    <details open><summary>Video guide: leveraging linked selection</summary>
        <iframe width="560" height="315" src="https://www.youtube.com/embed/TIwBlCH9YHw" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

    </details>

    <details open><summary>Video guide: active learning</summary>
        <iframe width="560" height="315" src="https://www.youtube.com/embed/hRIn3r7ovQ8" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

    </details>

    <details open><summary>Text guide: active learning</summary>
        Inspecting model predictions allows us to

        -   get an idea of how the current set of annotations will likely teach the model.
        -   locate the most valuable samples for further annotation.
    </details>

</details>